### Basic library imports

In [2]:
import os
import pandas as pd
import easyocr
import matplotlib.pyplot as plt
import cv2
import numpy as np
import csv

### Read Dataset

In [3]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [4]:
!python3 sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

/Users/goutham/Documents/GitHub/Amazon-hackathon/src/sanity.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
Parsing successfull for file: ../dataset/sample_test_out.csv


In [5]:
!python3 sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

/Users/goutham/Documents/GitHub/Amazon-hackathon/src/sanity.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
Error: Invalid unit [lbs] found in 6.75 lbs. Allowed units: {'microlitre', 'litre', 'fluid ounce', 'yard', 'cubic foot', 'foot', 'pound', 'centimetre', 'cubic inch', 'imperial gallon', 'metre', 'millivolt', 'kilogram', 'kilovolt', 'volt', 'quart', 'millimetre', 'watt', 'decilitre', 'cup', 'pint', 'millilitre', 'gallon', 'ounce', 'ton', 'kilowatt', 'inch', 'microgram', 'milligram', 'centilitre', 'gram'}


In [6]:
def load_image(image_path):
    return cv2.imread(image_path)

# Function to perform OCR and display results
def perform_ocr(image_path):
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(['en'])  # 'en' for English

    # Read the image
    image = load_image(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform OCR
    results = reader.readtext(image_rgb)

    # Create a copy of the image for drawing
    output_image = image_rgb.copy()

    # Plot the results
    # for (bbox, text, prob) in results:
    #     # Unpack the bounding box
    #     (tl, tr, br, bl) = bbox
    #     tl = (int(tl[0]), int(tl[1]))
    #     br = (int(br[0]), int(br[1]))

    #     # Draw the bounding box and text
    #     cv2.rectangle(output_image, tl, br, (0, 255, 0), 2)
    #     cv2.putText(output_image, text, (tl[0], tl[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # # Display the image with bounding boxes
    # plt.figure(figsize=(20,20))
    # plt.imshow(output_image)
    # plt.axis('off')
    # plt.show()

    # Print the extracted text
    #print("Extracted Text:")
    #for (bbox, text, prob) in results:
        #print(f"{text} (Confidence: {prob:.2f})")

    # Function to extract specific information (example for weight and voltage)
    def extract_info(results):
        info = {'weight': None, 'voltage': None}
        for (bbox, text, prob) in results:
            if 'g' in text.lower() or 'kg' in text.lower():
                info['weight'] = text
            elif 'v' in text.lower():
                info['voltage'] = text
        return info

    # Extract and print specific information
    extracted_info = extract_info(results)
    print("\nExtracted Information:")
    print(f"Weight: {extracted_info['weight']}")
    print(f"Voltage: {extracted_info['voltage']}")
    return results

### Download images

In [7]:
from utils import download_image
a=0
for i in train['image_link']:
    imgpath=download_image(i, save_folder='../images')
    extr=perform_ocr(imgpath)
    os.remove(imgpath)
    s = str(extr)
    print(s)
    with open("new.csv", mode='w',encoding='utf-8') as ft:
        ftwriter = csv.writer(ft)
        ftwriter.writerow(s)
    a+=1
    if a==100:
        break    


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(co

KeyboardInterrupt: 

: 

In [10]:
pwd()

'/Users/goutham/Documents/GitHub/Amazon-hackathon/src'

In [8]:
assert len(os.listdir('../images')) > 0

In [10]:
#rm -rf ../images